In [20]:
# API 키 임포트
import os

current_dir = os.getcwd()
key_path = os.path.join(current_dir, '..', '..', 'key', "vocal-entity-420406-b9648ba69fca.json")
key_path = os.path.abspath(key_path)
# 서비스 계정 키 파일의 경로를 환경 변수로 설정
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path


In [21]:
# 함수 정의
import io
import tkinter as tk
from tkinter import filedialog
from google.cloud import speech_v1p1beta1 as speech
from google.cloud import storage
import wave

def transcribe_audio(file_path):
    client = speech.SpeechClient()

    with io.open(file_path, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code="ko-KR",
    )

    with wave.open(file_path, 'rb') as wav_file:
        duration = wav_file.getnframes() / wav_file.getframerate()

    if duration > 60:
        print("Audio duration is longer than 1 minute, using long running recognize.")
        return transcribe_long_audio(file_path)
    else:
        response = client.recognize(config=config, audio=audio)

        transcripts = [result.alternatives[0].transcript for result in response.results]
        return transcripts

def transcribe_long_audio(file_path):
    client = speech.SpeechClient()
    
    bucket_name = 'stt_test_by_pm4'
    destination_blob_name = os.path.basename(file_path)
    gcs_uri = upload_to_gcs(bucket_name, file_path, destination_blob_name)

    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=48000,
        language_code="ko-KR",
    )
    operation = client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=180)
    
    transcripts = [result.alternatives[0].transcript for result in response.results]
    return transcripts

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    return f"gs://{bucket_name}/{destination_blob_name}"

def open_file_dialog():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    return file_path


In [22]:
# STT
file_path = open_file_dialog()
if file_path:
    result = transcribe_audio(file_path)
    
    print(result)


Audio duration is longer than 1 minute, using long running recognize.


InvalidArgument: 400 sample_rate_hertz (16000) in RecognitionConfig must either be omitted or match the value in the WAV header (48000).